In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import spacy
from gensim.models import word2vec
from sklearn.feature_extraction.text import CountVectorizer

import tensorflow as tf
from tensorflow import keras

!python -m spacy download fr_core_news_md

     |████████████████████████████████| 88.6 MB 1.2 MB/s 
  Created wheel for fr-core-news-md: filename=fr_core_news_md-2.2.5-py3-none-any.whl size=90338488 sha256=d1f6fab7315c40084ef9d6670947f41a6b0284b3b41d0d1cc02e3cc665a24284
  Stored in directory: /tmp/pip-ephem-wheel-cache-e03sh4o3/wheels/2e/26/ff/ce93eb966e7176ebe81e6c98209582e13e108cdd2d6d636df0
Successfully built fr-core-news-md
✔ Download and installation successful
You can now load the model via spacy.load('fr_core_news_md')


In [ ]:
data = pd.read_csv('training_data.csv')
data.drop(columns = ['id'], inplace = True)
data.head()

FileNotFoundError: ignored

In [ ]:
def type_counter(df):
    nlp = spacy.load('fr_core_news_md')
    dif_typ = []
    
    for i in range(df.shape[0]):
        doc = nlp(df['sentence'].iloc[i])
        for j in doc:
            if j.pos_ not in dif_typ:
                dif_typ.append(j.pos_)
                
    res = []
    
    for i in range(df.shape[0]):
        temp = np.zeros(len(dif_typ))
        doc = nlp(df['sentence'].iloc[i])
        for j in doc:
            ind = dif_typ.index(j.pos_)
            temp[ind] += 1
        res.append(temp)
        
                
    return res

In [ ]:
def sk_vec(df):
    phrases = df['sentence'].tolist()
    vectorizer = CountVectorizer()
    return vectorizer.fit_transform(phrases).toarray()

In [ ]:
def vectorizer(df):
    tokenized_sentence = []
    nlp = spacy.load('fr_core_news_md')
    for sentence in df['sentence'].tolist():
        doc = nlp(sentence)
        tokenized_sentence.append(doc.vector)
        
    return tokenized_sentence            

In [ ]:
def word_len_counter(df):
    res = []
    nlp = spacy.load('fr_core_news_md')
    for sentence in df['sentence'].tolist():
        temp = []
        doc = nlp(sentence)
        #doc=nb mots dans une phrase (ponct comprise)
        temp.append(len(doc))
        #sentence = nb de caractères dans une phrase(ponct comprise)
        temp.append(len(sentence))
        res.append(temp)
    return res
        

In [ ]:
def prepare_output(df):
    res1 = df['difficulty'].map({'A1':0, 'A2':1, 'B1':2,'B2':3,'C1':4,'C2':5}).tolist()
    
    res2 = []
    mapping =  {'A1':0, 'A2':1, 'B1':2,'B2':3,'C1':4,'C2':5}
    for i in range(len(df)):
        temp = np.zeros(6)
        temp[mapping[df['difficulty'].iloc[i]]] = 1
        res2.append(temp)
        
    
    return res1, res2

In [ ]:
def tf_accuracy(model, X,y):
    pred = model.predict(X)
    correct = 0
    for i in range(len(pred)):
        val =np.argmax(pred[i])
        if y[i][val] == 1:
            correct +=1
    return correct/len(y)

In [ ]:
feature1 = type_counter(data)

In [ ]:
feature2 = vectorizer(data)

In [ ]:
feature3 = word_len_counter(data)

In [ ]:
feature4 = sk_vec(data)

In [ ]:
feature = np.concatenate((feature1, feature2),axis = 1)
feature = np.concatenate((feature, feature3), axis = 1)
feature = np.concatenate((feature, feature4), axis = 1)

In [ ]:
y1, y2 = prepare_output(data)

In [ ]:
print(y1, y2)

In [ ]:
#X_train
feature1_train = np.array(feature[:4000])
y2_train = np.asarray(y2[:4000])
y1_train = np.asarray(y1[:4000])
#X_test
feature1_test = np.array(feature[4000:])
y2_test = np.asarray(y2[4000:])
y1_test = np.asarray(y1[4000:])

In [ ]:
model = keras.Sequential([
    keras.layers.Dense(input_shape = feature1_train[0].shape, units=512, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(units=6, activation='softmax')
])

print(model.summary())

In [ ]:
model.compile(optimizer='adamax', 
              loss=tf.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

In [ ]:
model.fit(feature1_train, y2_train, epochs = 100, steps_per_epoch = 500, validation_split=0.01)

In [ ]:
tf_accuracy(model, feature1_test, y2_test)

In [ ]:
def evaluate(true, pred):
    precision = precision_score(true, pred, average="weighted")
    recall = recall_score(true, pred, average="weighted")
    f1 = f1_score(true, pred, average="weighted")
    print(confusion_matrix(true, pred))
    print(accuracy_score(true, pred))
    print(precision, recall, f1)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

classifier = LogisticRegression()
classifier=classifier.fit(feature1_train, y1_train)


y_pred=classifier.predict(feature1_test)
for i in y_pred:
  temp=None
  

evaluate(y1_test, y_pred)



In [ ]:
from sklearn.neighbors import KNeighborsClassifier


classifier = KNeighborsClassifier()
classifier=classifier.fit(feature1_train, y1_train)


y_pred=classifier.predict(feature1_test)
  

evaluate(y1_test, y_pred)



In [ ]:
from sklearn.tree import DecisionTreeClassifier

classifier = DecisionTreeClassifier()
classifier=classifier.fit(feature1_train, y1_train)


y_pred=classifier.predict(feature1_test)
  

evaluate(y1_test, y_pred)

from sklearn.ensemble import RandomForestClassifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier()
classifier=classifier.fit(feature1_train, y1_train)


y_pred=classifier.predict(feature1_test)
  

evaluate(y1_test, y_pred)



In [ ]:
model = keras.Sequential([
    keras.layers.Dense(input_shape = feature1_train[0].shape, units=512, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(units=6, activation='softmax')
])

print(model.summary())
model.compile(optimizer='adamax', 
              loss=tf.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])
model.fit(np.asarray(feature), np.asarray(y2), epochs = 100, steps_per_epoch = 500, validation_split=0.01)